*All props go to https://colab.research.google.com/github/ozanpekmezci/capstone/blob/master/prepare-svhn.ipynb#scrollTo=F0_IdGzKqrQL*

# Prepare SVHN Dataset
This notebook serves to download and preprocess SVHN data, which will be used by other notebooks to classify street numbers.
## Download, Extraction & Preprocess SVHN Data


In [48]:
# These are all the modules we'll be using later.
from __future__ import print_function
import numpy as np

# from six.moves import cPickle as pickle
# from six.moves import range

import os
import sys
import tarfile
from IPython.display import display, Image

import h5py

import matplotlib.pyplot as plt

from PIL import Image
import random
from urllib.request import urlretrieve

In [49]:
from collections import Counter

In [50]:
%matplotlib inline

In [51]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Download dataset 

In [41]:
url = 'http://ufldl.stanford.edu/housenumbers/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
    """A hook to report the progress of a download. This is mostly intended for users with
    slow internet connections. Reports every 1% change in download progress.
    """
    global last_percent_reported
    percent = int(count * blockSize * 100 / totalSize)

    if last_percent_reported != percent:
        if percent % 5 == 0:
            sys.stdout.write("%s%%" % percent)
            sys.stdout.flush()
        else:
            sys.stdout.write(".")
            sys.stdout.flush()

        last_percent_reported = percent

def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    if force or not os.path.exists(filename):
        print('Attempting to download:', filename)
        filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
        print('\nDownload Complete!')
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

train_filename = maybe_download('train.tar.gz', 404141560 )
test_filename = maybe_download('test.tar.gz', 276555967 )

Found and verified train.tar.gz
Found and verified test.tar.gz


### Extracting Data

In [42]:
num_classes = 10
np.random.seed(133)

train_filename = 'train.tar.gz'
test_filename = 'test.tar.gz'
# extra_filename = 'extra.tar.gz'

def maybe_extract(filename, force=False):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s.' % (root, filename))
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
    if not os.path.exists(root+'/digitStruct.mat'):
        print("digitStruct.mat is missing")
    return root + '/digitStruct.mat'
  
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for train. This may take a while. Please wait.
Extracting data for test. This may take a while. Please wait.


### Read the dataset

In [43]:
print(train_folders)

train/digitStruct.mat


In [44]:
train_dataset = h5py.File(train_folders, "r")

In [45]:
train_dataset["digitStruct"].items()

ItemsViewHDF5(<HDF5 group "/digitStruct" (2 members)>)

In [46]:
# The DigitStructFile is just a wrapper around the h5py data.  It basically references 
#    inf:              The input h5 matlab file
#    digitStructName   The h5 ref to all the file names
#    digitStructBbox   The h5 ref to all struc data
class DigitStructFile:
    def __init__(self, inf):
        self.inf = h5py.File(inf, 'r')
        self.digitStructName = self.inf['digitStruct']['name']
        self.digitStructBbox = self.inf['digitStruct']['bbox']

# getName returns the 'name' string for for the n(th) digitStruct. 
    def getName(self,n):
        return ''.join([chr(c[0]) for c in self.inf[self.digitStructName[n][0]][()]])

# bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox. 
    def bboxHelper(self,attr):
        if (len(attr) > 1):
            attr = [self.inf[attr[()][j].item()][()][0][0] for j in range(len(attr))]
        else:
            attr = [attr[()][0][0]]
        return attr

# getBbox returns a dict of data for the n(th) bbox. 
    def getBbox(self,n):
        bbox = {}
        bb = self.digitStructBbox[n].item()
        bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
        bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
        bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
        bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
        bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
        return bbox

    def getDigitStructure(self,n):
        s = self.getBbox(n)
        s['name']=self.getName(n)
        return s

# getAllDigitStructure returns all the digitStruct from the input file.     
    def getAllDigitStructure(self):
        return [self.getDigitStructure(i) for i in range(len(self.digitStructName))]

# Return a restructured version of the dataset (one structure by boxed digit).
#
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#          'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#          'left', 'top' : position of bounding box
#          'width', 'height' : dimension of bounding box
#
# Note: We may turn this to a generator, if memory issues arise.
    def getAllDigitStructure_ByDigit(self):
        pictDat = self.getAllDigitStructure()
        result = []
        structCnt = 1
        for i in range(len(pictDat)):
            item = { 'filename' : pictDat[i]["name"] }
            figures = []
            for j in range(len(pictDat[i]['height'])):
                figure = {}
                figure['height'] = pictDat[i]['height'][j]
                figure['label']  = pictDat[i]['label'][j]
                figure['left']   = pictDat[i]['left'][j]
                figure['top']    = pictDat[i]['top'][j]
                figure['width']  = pictDat[i]['width'][j]
                figures.append(figure)
            structCnt = structCnt + 1
            item['boxes'] = figures
            result.append(item)
        return result

In [47]:
print(train_folders)
train = DigitStructFile(train_folders)
train_data = train.getAllDigitStructure_ByDigit()


test = DigitStructFile(test_folders)
test_data = test.getAllDigitStructure_ByDigit()

train/digitStruct.mat


In [52]:
print('Train objects:', len(train_data))
print('Test objects:', len(test_data))

Train objects: 33402
Test objects: 13068


In [53]:
train_data[20000]

{'filename': '20001.png',
 'boxes': [{'height': 58.0,
   'label': 1.0,
   'left': 172.0,
   'top': 108.0,
   'width': 18.0},
  {'height': 58.0, 'label': 3.0, 'left': 185.0, 'top': 105.0, 'width': 26.0},
  {'height': 58.0, 'label': 6.0, 'left': 208.0, 'top': 98.0, 'width': 24.0}]}

### Getting image dimensions and preparing dataset in YOLO format

First use something like
```
for f in *.png; do mv -- "$f" "${f%.txt}_.png"; done
```
on the `test` images so that names of the images would not overlap.

In [54]:
from get_image_size import get_image_size

def make_labels(data, folder_name, is_test=False):
    for data_element in data:
        width, height = get_image_size(os.path.join(folder_name, data_element['filename'].replace('.png', '')+('_' if is_test else '')+'.png'))
        with open(os.path.join('labels', folder_name, data_element['filename'].replace('.png', '')+('_' if is_test else '')+'.txt'), 'w') as label_file:
            for number_item in data_element['boxes']:
                label_file.write(
                    str(int(number_item['label']) % 10)+' '+ \
                    str((number_item['left'] + number_item['width'] / 2) / width)+' '+ \
                    str((number_item['top'] + number_item['height'] / 2) / height)+' '+ \
                    str(number_item['width'] / width)+' '+ \
                    str(number_item['height'] / height)+' '+ \
                    '\n'
                )

make_labels(train_data, 'train')
make_labels(test_data, 'test', True)